import json
from glob import glob

In [2]:
import json
from glob import glob
import numpy as np
import re
from collections import defaultdict

In [71]:
# Opening JSON file
def extract_numbers(test_string):
    test_string = test_string.replace('<h-l-1>', '')
    test_string = test_string.replace('</h-l-1>', '')

    test_string = test_string.replace('<h-l-2>', '')

    test_string = test_string.replace('</h-l-2>', '')

    test_string = test_string.replace('<h-l-3>', '')
    test_string = test_string.replace('</h-l-3>', '')

    numbers = re.findall(r'\d+', str(test_string))
    return numbers

batches = ['0']
usefuleness, conf, like = [], [], []
reliance = []
divergence, div_easy, original = [], [],[]
divergence_type = []
a_ =defaultdict(list)

sorted_annotations = defaultdict(list)
time_without, time_with = [],[]
sorted_anno = defaultdict(list)
sorted_anno_false = defaultdict(list)
lang = 'es'

for batch in batches: 
    print(batch)
    for f_name in glob(f'responses/{lang}/*.json'):
        critical_hard_true, critical_hard_false, critical_easy_true, critical_easy_false = 0,0,0,0
        if 'marine' in f_name:
            continue
        print(f_name)
        #usefuleness, conf, like = [], [], []
        reliance = []
        div_easy =[]
        divergence, original = [], []
        divergence_type = []
        time_without, time_with = [],[]
        f = open(f_name)
        data = json.load(f)
        try:
            if data['language'] == 'Limited':
                a=1
                continue
        except:
            a=1


        if data['with_hl'] == 'false':
            a=1
        print(data['with_hl'])

        conf.append(float(data['r28']))
        if data['with_hl'] == 'false':
            time_with.append(data['totalTimeInMinutes'])
        else:
            time_without.append(data['totalTimeInMinutes'])

        try:
            usefuleness.append(float(data['h28']))
        except:
            a = 1
        
        try:
            if data['with_hl'] == 'true':
                like.append(float(data['l28']))
        except:
            a=1
            
        inds = [a for a in data['sample_indices'].split(',')]
        for id_, ind_ in enumerate(inds):
            ind = str(ind_)
            with open(f'static/examples/for_user_study/flores_gpt_paraphrase_{lang}/meta_lbl_{ind}.txt', 'r') as meta:
                meta = meta.readlines()[0]

                try:
                    anno = data[f't{id_+1}']
                    
                    if data['with_hl'] == 'true': 
                        if meta == 'chatGPT_paraphrase':
                            sorted_anno[f'B_{ind_}'].append(data[f'class{id_+1}'])
                        elif meta == 'chatGPT_divergence':
                            sorted_anno[f'A_{ind_}'].append(data[f'class{id_+1}'])
                        else:
                            sorted_anno[f'C_{ind_}'].append(data[f'class{id_+1}'])
                    else:
                        if meta == 'chatGPT_paraphrase':
                            sorted_anno_false[f'B_{ind_}'].append(data[f'class{id_+1}'])
                        elif meta == 'chatGPT_divergence':
                            sorted_anno_false[f'A_{ind_}'].append(data[f'class{id_+1}'])
                        else:
                            sorted_anno_false[f'C_{ind_}'].append(data[f'class{id_+1}'])

                except:
                    if data['with_hl'] == 'true': 

                        if meta == 'chatGPT_paraphrase':
                            sorted_anno[f'B_{ind_}'].append('Missing')
                        elif meta == 'chatGPT_divergence':
                            sorted_anno[f'A_{ind_}'].append('Missing')
                        else:
                            sorted_anno[f'C_{ind_}'].append('Missing')
                    else:
                        if meta == 'chatGPT_paraphrase':
                            sorted_anno_false[f'B_{ind_}'].append('Missing')
                        elif meta == 'chatGPT_divergence':
                            sorted_anno_false[f'A_{ind_}'].append('Missing')
                        else:
                            sorted_anno_false[f'C_{ind_}'].append('Missing')                        
                    
                    print('Missing')
                    print(meta)
                    continue

                if meta=='chatGPT_divergence':#  anno=='no':
                    
                    with open(f'static/examples/for_user_study/flores_gpt_paraphrase_{lang}/src_{ind}.txt', 'r') as src:
                        src = src.readlines()[0]
                        src_num = extract_numbers(src)

                    with open(f'static/examples/for_user_study/flores_gpt_paraphrase_{lang}/tgt_{ind}.txt', 'r') as tgt:
                        tgt = tgt.readlines()[0]
                        tgt_num = extract_numbers(tgt)
                    
 
                type_ = data[f'class{id_+1}']
                if type_ !='N/A':# or 'Minor' in type_:
                    anno_corrected = 'yes'
                    if data['with_hl'] == 'false': 
                        if meta == 'chatGPT_divergence':
                            critical_hard_false +=1
                        elif meta =='ced':
                            critical_easy_false +=1
                    else:
                        if meta == 'ced':
                            critical_easy_true +=1
                        elif meta =='chatGPT_divergence':
                            critical_hard_true +=1 
                            
                            
                #elif 'Minor' in type_:
                #    anno_corrected = 'no'
                #    if data['with_hl'] == 'false': 
                #        if meta == 'ced':
                #            critical_easy_false +=1   
                #    else:
                #        if meta =='ced':
                #            critical_easy_true +=1

        
                else:
                    anno_corrected = 'no'
                if meta == 'chatGPT_divergence':
                    divergence.append(anno_corrected)
                    divergence_type.append(data[f'class{id_+1}'])
                elif meta == 'ced':
                    div_easy.append(anno_corrected)
                else:
                    original.append(anno_corrected)

                    
        d_yes_h, d = divergence.count('yes'), len(divergence)
        d_yes_e, d = div_easy.count('yes'), len(div_easy)
        o_no, o = original.count('no'), len(original)

        tp = d_yes_h + d_yes_e
        fp = original.count('yes')
        fn = divergence.count('no') + div_easy.count('no')

        #precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        u = np.mean(usefuleness)
        c = np.mean(conf)
        l = np.mean(like)
        print(recall)

        if data['with_hl'] == 'false': 
            #recall = critical_hard_false/5
            print(f'Critical Hard: {critical_hard_false}')
            print(f'Critical Easy: {critical_easy_false}')
        else:
            #recall = critical_hard_true/5
            print(f'Critical Hard: {critical_hard_true}')
            print(f'Critical Easy: {critical_easy_true}')
        try:
            en = data['language']
            print(f'EN: {en}')
        except:
            a =1
        print('\n')
        a_[data['with_hl']].append(d_yes_h/5)
        print(sorted_anno)
        sorted_annotations[data['with_hl']].append([recall, list(dict(sorted(sorted_anno.items())).values())])

0
responses/es/20230421-103120.json
true
0.8
Critical Hard: 8
Critical Easy: 0


defaultdict(<class 'list'>, {'B_50': ['N/A'], 'A_48': ['Changed'], 'C_30': ['N/A'], 'B_45': ['Added (in English)'], 'C_33': ['N/A'], 'A_35': ['Added (in English)'], 'C_32': ['Changed'], 'B_28': ['N/A'], 'A_40': ['Added (in Spanish)'], 'B_46': ['N/A'], 'C_16': ['N/A'], 'C_9': ['N/A'], 'C_20': ['N/A'], 'C_38': ['N/A'], 'A_23': ['Added (in English)'], 'A_2': ['Added (in English)'], 'A_12': ['N/A'], 'C_39': ['N/A'], 'A_34': ['Added (in Spanish)'], 'C_26': ['N/A'], 'C_27': ['N/A'], 'C_10': ['N/A'], 'A_43': ['N/A'], 'A_47': ['Added (in English)'], 'A_15': ['Added (in Spanish)']})
responses/es/20230421-103828.json
false
1.0
Critical Hard: 8
Critical Easy: 0


defaultdict(<class 'list'>, {'B_50': ['N/A'], 'A_48': ['Changed'], 'C_30': ['N/A'], 'B_45': ['Added (in English)'], 'C_33': ['N/A'], 'A_35': ['Added (in English)'], 'C_32': ['Changed'], 'B_28': ['N/A'], 'A_40': ['Added (in Spanish)'], 'B_46': ['N/A'], 'C_16'

0.6666666666666666
Critical Hard: 8
Critical Easy: 0


defaultdict(<class 'list'>, {'B_50': ['N/A', 'N/A'], 'A_48': ['Changed', 'Added (in Spanish)'], 'C_30': ['N/A', 'N/A'], 'B_45': ['Added (in English)', 'Changed'], 'C_33': ['N/A', 'N/A'], 'A_35': ['Added (in English)', 'Added (in English)'], 'C_32': ['Changed', 'N/A'], 'B_28': ['N/A', 'N/A'], 'A_40': ['Added (in Spanish)', 'N/A'], 'B_46': ['N/A', 'Added (in Spanish)'], 'C_16': ['N/A', 'N/A'], 'C_9': ['N/A', 'N/A'], 'C_20': ['N/A', 'Changed'], 'C_38': ['N/A', 'N/A'], 'A_23': ['Added (in English)', 'Added (in English)'], 'A_2': ['Added (in English)', 'Added (in English)'], 'A_12': ['N/A', 'N/A'], 'C_39': ['N/A', 'N/A'], 'A_34': ['Added (in Spanish)', 'Added (in Spanish)'], 'C_26': ['N/A', 'N/A'], 'C_27': ['N/A', 'N/A'], 'C_10': ['N/A', 'N/A'], 'A_43': ['N/A', 'Changed'], 'A_47': ['Added (in English)', 'Added (in English)'], 'A_15': ['Added (in Spanish)', 'Added (in Spanish)'], 'B_44': ['N/A', 'N/A'], 'C_7': ['N/A', 'N/A'], 'C_18': ['A

In [72]:
sorted(sorted_anno.items())

[('A_12', ['N/A', 'N/A', 'Changed']),
 ('A_15', ['Added (in Spanish)', 'Added (in Spanish)', 'Added (in Spanish)']),
 ('A_19', ['Added (in English)', 'Changed', 'Added (in Spanish)']),
 ('A_2', ['Added (in English)', 'Added (in English)', 'Added (in English)']),
 ('A_23', ['Added (in English)', 'Added (in English)', 'Added (in English)']),
 ('A_25', ['Added (in English)', 'Added (in English)', 'Changed']),
 ('A_3', ['Added (in Spanish)', 'Added (in Spanish)', 'Added (in Spanish)']),
 ('A_34', ['Added (in Spanish)', 'Added (in Spanish)', 'Added (in Spanish)']),
 ('A_35', ['Added (in English)', 'Added (in English)', 'Added (in English)']),
 ('A_36', ['Changed', 'N/A', 'Changed']),
 ('A_4', ['N/A', 'N/A', 'Changed']),
 ('A_40', ['Added (in Spanish)', 'N/A', 'Changed']),
 ('A_41', ['Changed', 'N/A', 'Changed']),
 ('A_42', ['Changed', 'Changed', 'Changed']),
 ('A_43', ['N/A', 'Changed', 'Changed']),
 ('A_47', ['Added (in English)', 'Added (in English)', 'Added (in English)']),
 ('A_48', ['C

In [73]:
import numpy as np
#recalls, annos = [], []
#for a in sorted_annotations['true']:
#    recalls.append(a[0])
#    annos.append(a[1:])
#order_ = list(np.argsort(recalls))[::-1]

#hl = []
    
#for o in order_:
#    print(o)
#    print(recalls[o])
#    print(annos[o])
sorted_anno_ = dict(sorted(sorted_anno.items()))

#for a in sorted_annotations['true']:
for id_, key in enumerate(sorted_anno_.keys()):
    b = []
    for i, x in enumerate(sorted_anno_false[key]):
        if 'Changed' in x:
            b.append('\cellcolor{red!10} \dive')
        elif 'Added' in x:
            b.append('\cellcolor{red!10} \dive')
        else:
            b.append('\cellcolor{green!10} \equiva')
    for i, x in enumerate(sorted_anno[key]):
        if 'Changed' in x:
            b.append('\cellcolor{red!10} \dive')
        elif 'Added' in x:
            b.append('\cellcolor{red!10} \dive')
        else:
            b.append('\cellcolor{green!10} \equiva')
            
        
    c = ' & '.join(b)
    print(c + ' \\\\')
    print('%')
    if id_ ==20 or id_ == 29:
        print(' \\addlinespace[2em]')


    
print('\\\\')    



\cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{green!10} \equiva & \cellcolor{green!10} \equiva & \cellcolor{red!10} \dive \\
%
\cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive \\
%
\cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive \\
%
\cellcolor{green!10} \equiva & \cellcolor{green!10} \equiva & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive \\
%
\cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive \\
%
\cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive & \cellcolor{red!10} \dive

In [74]:
batch = '0'
for i in range(1,25):
    with open(f'static/examples/for_user_study/gpt_lit_en_pt_ced_{batch}/src_{i}.txt', 'r') as src:
        src = src.readlines()[0]
        
    with open(f'static/examples/for_user_study/gpt_lit_en_pt_ced_{batch}/tgt_{i}.txt', 'r') as tgt:
        tgt = tgt.readlines()[0]
        
        if 'h-l-' in src or 'h-l-' in tgt:
            print(f'{i} - HIGH')

FileNotFoundError: [Errno 2] No such file or directory: 'static/examples/for_user_study/gpt_lit_en_pt_ced_0/src_1.txt'

In [68]:
dict(sorted_anno_)

{'A_12': ['N/A', 'Added (in English)', 'N/A'],
 'A_15': ['N/A', 'Added (in French)', 'Added (in French)'],
 'A_19': ['Changed', 'Changed', 'Added (in French)'],
 'A_2': ['Added (in English)', 'Added (in French)', 'Added (in English)'],
 'A_23': ['Changed', 'Added (in English)', 'Added (in English)'],
 'A_26': ['Added (in English)', 'Changed', 'Added (in English)'],
 'A_3': ['N/A', 'Added (in French)', 'Added (in French)'],
 'A_35': ['Added (in French)', 'Added (in French)', 'Added (in French)'],
 'A_38': ['Added (in English)', 'Added (in English)', 'Added (in English)'],
 'A_39': ['N/A', 'N/A', 'N/A'],
 'A_4': ['Added (in French)', 'Changed', 'Added (in French)'],
 'A_41': ['Added (in French)', 'Changed', 'Added (in French)'],
 'A_42': ['N/A', 'N/A', 'Added (in French)'],
 'A_43': ['Changed', 'Changed', 'Added (in French)'],
 'A_44': ['Changed', 'Changed', 'Added (in French)'],
 'A_48': ['Missing', 'Added (in English)', 'Changed'],
 'A_49': ['Added (in French)', 'Added (in French)', 'A